In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
DATA_PATH = "./../data/use_case_andalusia/metadata"

In [ ]:
Product_Groups = pd.read_csv(os.path.join(DATA_PATH, "Product_Groups.csv"))
display(Product_Groups)

In [ ]:
crops_codes = pd.read_csv(os.path.join(DATA_PATH, "crops_codes.csv"))
display(crops_codes)

In [ ]:
product_mapping_esp = pd.DataFrame()
product_mapping_esp["Description"] = crops_codes["Description"]
product_mapping_esp["FADN Included products"] = crops_codes["Description"]
product_mapping_esp["FADN Included products IDs"] = crops_codes["code"]
product_mapping_esp["CUSTOM GROUP (EN)"] = np.nan

product_mapping_esp["FADN Included products IDs"] = product_mapping_esp["FADN Included products IDs"].astype(str)
product_mapping_esp

In [ ]:
product_mapping_ita = pd.read_csv(os.path.join(DATA_PATH, "Product_Mapping_ita.csv"))
display(product_mapping_ita)


In [ ]:
product_mapping_ita["CUSTOM GROUP (EN)"].unique()

In [ ]:
for idx in product_mapping_esp.index:
    code = product_mapping_esp.at[idx, "FADN Included products IDs"]
    
    if code in product_mapping_ita["FADN Included products IDs"].dropna().unique():
        print(code)
        # Assign custom group
        custom_group = [cg for cg in product_mapping_ita[product_mapping_ita["FADN Included products IDs"]==code]["CUSTOM GROUP (EN)"].dropna().unique() if not cg.startswith("ORG")]
        if len(custom_group)>0:
            product_mapping_esp.at[idx, "CUSTOM GROUP (EN)"] = custom_group[0]
            print("    ", custom_group)

In [ ]:
product_mapping_esp["product_code"] = product_mapping_esp["FADN Included products IDs"].apply(lambda x: x)

display(product_mapping_esp)

In [29]:
product_mapping_esp["CUSTOM GROUP (EN)"] = product_mapping_esp.apply(lambda x: x["CUSTOM GROUP (EN)"] if not pd.isna(x["CUSTOM GROUP (EN)"]) else "OTHER", axis=1)

#product_mapping_esp["CUSTOM GROUP (EN)"] = product_mapping_esp["CUSTOM GROUP (EN)"].apply(lambda x: x if isinstance(x, "str") else "OTHER")

In [ ]:
product_mapping_esp["CUSTOM GROUP (EN)"].unique()

In [ ]:
# Check that all the codes available in ITA use case have been assigned in the SPA use case
product_mapping_ita[product_mapping_ita["FADN Included products IDs"].isin(product_mapping_esp[product_mapping_esp["CUSTOM GROUP (EN)"].isna()]["FADN Included products IDs"].tolist())]


In [32]:
# Generate organic crops
product_mapping_esp_org = product_mapping_esp.copy(deep=True)

In [ ]:
product_mapping_esp

In [34]:
product_mapping_esp_org["CUSTOM GROUP (EN)"] = product_mapping_esp_org["CUSTOM GROUP (EN)"].dropna().apply(lambda x: f'ORG_{x}')

In [ ]:
product_mapping_esp = pd.concat([product_mapping_esp, product_mapping_esp_org]).reset_index(drop=True)
display(product_mapping_esp)

In [36]:
# Save Spanish product mapping
FILENAME = "Product_Mapping_esp.csv"
product_mapping_esp.to_csv(os.path.join(DATA_PATH, FILENAME), index=False)